In [1]:
import os
import math
import config
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import (
    create_img_grid,
    SpatialProjection
)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
data_dir = "../../data/dataset/raw/"
images_dir = "../../data/dataset/images/"
subjects = os.listdir(data_dir)

IMG_LEN = 512

sp = SpatialProjection(
    img_dir="../../data/dataset/images/",
    img_len=math.floor(IMG_LEN / 3),
    polyfit_degree=0
)


In [3]:
images = []
labels = []

for subject in subjects:
    for gesture in config.GESTURES:
        gesture_dir = os.path.join(data_dir, subject, gesture)

        recordings = []
        try:
            recordings = os.listdir(gesture_dir)
        except FileNotFoundError:
            continue

        for recording in recordings:
            _images = []
            file_path = os.path.join(gesture_dir, recording)

            data = pd.read_csv(file_path)
            data.drop(columns=["time"], inplace=True)
            data.drop(0, inplace=True)  # Remove first All-0 row

            for landmark in config.PROJECTION_LANDMARKS:
                _images.extend(
                    sp.get_projection_images(
                        data=data.filter(regex=landmark),
                        subject=subject,
                        gesture=gesture
                    )
                )

            img = create_img_grid(_images, IMG_LEN)

            images.append(img)
            labels.append(config.GESTURES.index(gesture))

            plt.imshow(img)
            plt.title(f"{subject}: {gesture}")
            save_path = os.path.join(images_dir, subject, gesture)
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            plt.savefig(os.path.join(save_path, recording + ".jpg"))
            plt.close()
